In [1]:
# IMPORTS
# Standard
import os
import pathlib

# Third Party
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import torch

# Interpretability
from interpretability.interpretability_models import symbolic_pursuit_explainer
from interpretability.interpretability_models.utils import io
from interpretability.models.multilayer_perceptron import DiabetesMLPRegressor

In [2]:

X, y = load_diabetes(return_X_y=True, as_frame=True)

feature_names = X.columns.to_list()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train, X_test, y_train, y_test = (
    X_train.to_numpy(),
    X_test.to_numpy(),
    y_train.to_numpy(),
    y_test.to_numpy(),
)

### Add this saved model to google drive

In [3]:
## Load the model

model = DiabetesMLPRegressor()  # pytorch MLP



def load_trained_model(model, trained_model_state_path):
    model.load_state_dict(torch.load(trained_model_state_path))
    model.eval()
    return model


desktop_path = pathlib.Path.home() / 'Desktop'

TRAINED_MODEL_STATE_PATH = os.path.join(desktop_path, "model_cv1.pth")
model = load_trained_model(model, TRAINED_MODEL_STATE_PATH)



RuntimeError: Error(s) in loading state_dict for DiabetesMLPRegressor:
	Missing key(s) in state_dict: "lin1.weight", "lin1.bias", "lin2.weight", "lin2.bias", "lin3.weight", "lin3.bias". 
	Unexpected key(s) in state_dict: "convInput.weight", "convInput.bias", "convHidden1.weight", "convHidden1.bias", "convHidden2.weight", "convHidden2.bias", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "bn1.num_batches_tracked", "bn2.weight", "bn2.bias", "bn2.running_mean", "bn2.running_var", "bn2.num_batches_tracked", "bn3.weight", "bn3.bias", "bn3.running_mean", "bn3.running_var", "bn3.num_batches_tracked", "fc1.weight", "fc1.bias". 

In [ ]:

my_explainer = symbolic_pursuit_explainer.SymbolicPursuitExplainer(
    model.predict, X_test, feature_names=feature_names, loss_tol=0.01, patience=5
)

my_explainer.fit()

my_explainer.measure_fit_quality(X_test, y_test)

explanation = my_explainer.explain()

In [ ]:

my_explainer.summary_plot()

In [ ]:

io.save_explainer(
    my_explainer, "my_new_diabetes_pytorch_mlp_symbolic_pursuit_explainer.p"
)